In [4]:
from ast import Try
from PyQt5.QtWidgets import QLabel, QHBoxLayout, QVBoxLayout, QApplication, QWidget
from picamera2 import Picamera2
from PyQt5.QtGui import QImage,QPixmap, QPainter, QColor, QFont
from PyQt5.QtCore import QThread
import RPi.GPIO as gp
import time
import os

width = 640
height = 480 

recording_time = 10 # recording time in seconds

adapter_info = {  
    "A" : {   
        "i2c_cmd":"i2cset -y 10 0x70 0x00 0x04",
        "gpio_sta":[0,0,1],
        }, 
    "B" : {
        "i2c_cmd":"i2cset -y 10 0x70 0x00 0x05",
        "gpio_sta":[1,0,1],
        },
    "C" : {
        "i2c_cmd":"i2cset -y 10 0x70 0x00 0x06",
        "gpio_sta":[0,1,0],
        },
    #"D" : {
     #   "i2c_cmd":"i2cset -y 10 0x70 0x00 0x07",
     #   "gpio_sta":[1,1,0],
    #}
}

class WorkThread(QThread):

    def __init__(self):
        super(WorkThread,self).__init__()
        gp.setwarnings(False)
        gp.setmode(gp.BOARD)
        gp.setup(7, gp.OUT)
        gp.setup(11, gp.OUT)
        gp.setup(12, gp.OUT)


    def select_channel(self,index):
        channel_info = adapter_info.get(index)
        if channel_info == None:
            print("Can't get this info")
        gpio_sta = channel_info["gpio_sta"] # gpio write
        gp.output(7, gpio_sta[0])
        gp.output(11, gpio_sta[1])
        gp.output(12, gpio_sta[2])

    def init_i2c(self,index):
        channel_info = adapter_info.get(index)
        os.system(channel_info["i2c_cmd"]) # i2c write

    def grab_one_image(self):
        global picam2
        # picam2 = Picamera2()
        # picam2.configure( picam2.still_configuration(main={"size": (320, 240),"format": "BGR888"},buffer_count=1))
        
        # pre-allocate video
        

        flag = False

        for item in {"A", "B", "C"}: #,"B","C","D"}:
            try:
                self.select_channel(item)
                self.init_i2c(item)
                time.sleep(0.5) 
                if flag == False:
                    flag = True
                else :
                    picam2.close()
                    # time.sleep(0.5) 
                print("init1 "+ item)
                picam2 = Picamera2()
                picam2.configure(picam2.create_still_configuration(main={"size": (640, 480),"format": "BGR888"},buffer_count=2)) 
                picam2.start()
                time.sleep(2)
                picam2.capture_array(wait=False)
                time.sleep(0.1)
            except Exception as e:
                print("except: "+str(e))

        t = time.time()
        
        # while True:
            
        for item in {"A", "B", "C"}: #,"B","C","D"}:
            self.select_channel(item)
            time.sleep(0.02)
            try:
                #buf = picam2.capture_array()
                buf = picam2.capture_array()
                cvimg = QImage(buf, width, height,QImage.Format_RGB888)
                
                # painter = QPainter(cvimg)
                # painter.setPen(QColor(255, 255, 255))  # white color
                # painter.setFont(QFont("Arial", 20))
                # painter.drawText(10, 30, str(round(1/(time.time() - t), 2))) # coordinates for where to place the text
                # painter.end()
        
                pixmap = QPixmap(cvimg)
                if item == 'A':
                    image_label.setPixmap(pixmap)
                elif item == 'B':
                    image_label2.setPixmap(pixmap)
                elif item == 'C':
                    image_label3.setPixmap(pixmap)
            except Exception as e:
                print("capture_buffer: "+ str(e))
                
        # t = time.time()
    

        return cvimg
                    

picam2 = Picamera2()
work = WorkThread()

work.start()
work.quit()
picam2.close()

[0:32:48.296182392] [2308]  INFO Camera camera_manager.cpp:297 libcamera v0.0.5+83-bde9b04f
[0:32:48.333540874] [2522]  INFO RPI vc4.cpp:437 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media0 and ISP device /dev/media2
[0:32:48.333653688] [2522]  INFO RPI pipeline_base.cpp:1101 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
